# The CNS North Korea Missile Test Database

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us
import requests

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Download latest version of the database from [here](https://beyondparallel.csis.org/database-north-korean-provocations/)

In [5]:
url = "https://www.nti.org/wp-content/uploads/2022/06/north_korea_missile_test_database_2022_06_24-5.xlsx"

In [6]:
resp = requests.get(url)
output = open("data/raw/north_korea_missile_test_database.xls", "wb")
output.write(resp.content)
output.close()

#### Read data

In [28]:
src = pd.read_excel(
    "data/raw/north_korea_missile_test_database.xls", skiprows=1, parse_dates=["Date"]
)

In [30]:
src.columns = (
    src.columns.str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
)

In [31]:
src["year"] = src["date"].dt.year
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [32]:
src.head()

,f1,date,date_entered_updated,launch_time_(utc),missile_name,missile_type,launch_agency_authority,facility_name,facility_location,other_name,facility_latitude,facility_longitude,landing_location,apogee,distance_travelled,confirmation_status,test_outcome,additional_information,source(s),year,month,day
0,1,1984-04-09,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,Unknown,Unknown,Confirmed,Success,First known test by North Korea of a missile w...,http://www.nti.org/media/pdfs/north_korea_miss...,1984,4,Monday
1,2,1984-04-09,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,Unknown,Unknown,Confirmed,Success,NaN,http://www.nti.org/media/pdfs/north_korea_miss...,1984,4,Monday
2,3,1984-04-09,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,200 km,Unknown,Confirmed,Success,NaN,http://www.astronautix.com/g/gitdaeryung.html;...,1984,4,Monday
3,4,1984-09-01,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,Unknown,Unknown,Confirmed,Failure,NaN,"Joseph S. Bermudez, ""A History of Ballistic Mi...",1984,9,Saturday
4,5,1984-09-01,2016-12-23,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,200 km,Unknown,Confirmed,Failure,NaN,http://www.astronautix.com/g/gitdaeryung.html;...,1984,9,Saturday
